In [1]:
%reload_ext autoreload

In [2]:
# Imports

%load_ext autoreload
%autoreload 2

%matplotlib inline
import numpy as np
import random
import csv
import pandas as pd

from fastai.imports import *
from fastai.structured import *
from matplotlib import pyplot
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.metrics import f1_score,\
    accuracy_score, confusion_matrix,\
    precision_score, recall_score,\
    roc_curve, roc_auc_score,\
    cohen_kappa_score, mean_absolute_error,\
    precision_recall_curve, auc,\
    average_precision_score

set_plot_sizes(12,14,16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
import csv

sys.path.append('../lib')
import utils
import preprocessing

In [4]:
# Load in data

PATH = "../data/"
df_raw = pd.read_csv(f'{PATH}churn.csv', low_memory=False, 
                     parse_dates=['canceldate', 'licence_registration_date', 'golive_date'])

In [5]:
print(len(df_raw.loc[df_raw.churned == 0]), df_raw.loc[df_raw.churned == 0].shape)

df_processed = preprocessing.prepare_data(df_raw)
print(len(df_processed), df_processed.shape)

df_filtered = df_processed.loc[df_processed.churned == 0]
df_data, y_data, nas = proc_df(df_filtered, 'churned')
print(len(df_data), df_data.shape)

2780 (2780, 38)


../lib/preprocessing.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['annual_revenue'][dataframe['annual_revenue'] == 0] = dataframe['annual_revenue'].median()
/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


5889 (5889, 93)
2780 (2780, 92)


/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
joblib_file = "churn_model.pkl"

# Load from file
churn_model = joblib.load(joblib_file)

In [7]:
predictions = churn_model.predict(df_data)
probability = churn_model.predict_proba(df_data)[:,1]
len(probability), len(predictions), len(df_filtered)

(2780, 2780, 2780)

In [8]:
df_predictions = pd.DataFrame(predictions)
df_probability = pd.DataFrame(probability)

In [9]:
df_output = df_processed[df_processed.churned == 0].copy()

In [10]:
df_output['prediction'] = predictions
df_output['probability'] = probability

In [11]:
# Calculate churn concern %
churn_concern_count = len(df_output.loc[df_output['prediction'] == 1])
total_active = len(df_output)
churn_concern_percent = round((churn_concern_count / total_active) * 100, 2)
print(f"Churn concern count: {churn_concern_count} of {total_active} ({churn_concern_percent}%)")

Churn concern count: 70 of 2780 (2.52%)


In [17]:
# Sort churn concerns by probability
df_sorted = df_output.loc[df_output['prediction'] == 1].sort_values(by=['probability'], ascending=False)

In [18]:
df_output.annual_revenue_log.describe()

count    2780.000000
mean       10.904841
std         2.485439
min        -0.094311
25%         9.116422
50%        10.818493
75%        12.880757
max        17.742592
Name: annual_revenue_log, dtype: float64

In [54]:
df_output.annual_revenue_log.var(), df_output.annual_revenue_log.skew(), df_output.annual_revenue_log.mad(), df_output.annual_revenue_log.mode()



(6.177406819449559, -0.25333110321829916, 2.0570651569924894, 0    9.116422
 dtype: float64)

In [55]:
2**9.116422

555.0300220652385

In [50]:
# Calculate the % of ros that have a annual_revenue_log of +/- 1 standard diviation
COLUMN_NAME='annual_revenue_log'
feature_mean = df_output[COLUMN_NAME].mean()
feature_std = df_output[COLUMN_NAME].std()

for i in [1,2,3]:
    output = round(len(df_output[COLUMN_NAME].loc[(df_output[COLUMN_NAME] > (feature_mean - (std1 * i))) &
                                            (df_output[COLUMN_NAME] < (feature_mean + (std1 * i)))]) / len(df_output) * 100,2)
    print(output)

71.73
96.19
99.53


In [51]:
now = datetime.datetime.now()

for i in range(50):
    print(f"{now},{df_sorted.username.iloc[i]},{df_raw[df_raw.username == df_sorted.username.iloc[i]].url.iloc[0]},{df_sorted.annual_revenue_log.iloc[i]},{df_sorted.probability.iloc[i]}")
          
    #with open('../data/predictions.csv', 'w') as writeFile:
    #    writer = csv.writer(writeFile)
    #    writer.writerows(sorted_array)

#writeFile.close()

2019-10-15 00:37:53.712119,N061491,marshall.neto.com.au,9.116421923264706,0.6897080815118563
2019-10-15 00:37:53.712119,N061711,lukasz-credit-t.neto.com.au,9.848569803002546,0.6838500325406136
2019-10-15 00:37:53.712119,N063834,urban-crush.neto.com.au,9.116421923264706,0.6830457621044072
2019-10-15 00:37:53.712119,N062610,hellotest.neto.com.au,12.818522329088832,0.6351416033187537
2019-10-15 00:37:53.712119,N061143,bmx-internation.neto.com.au,9.116421923264706,0.6308593974115767
2019-10-15 00:37:53.712119,N061402,zoomautoparts.neto.com.au,9.116421923264706,0.6293132035368315
2019-10-15 00:37:53.712119,N061328,la-luna-pet-car.neto.com.au,9.837109473782805,0.6279924881992694
2019-10-15 00:37:53.712119,N061504,supplychain.neto.com.au,9.116421923264706,0.6194579885874527
2019-10-15 00:37:53.712119,N063884,prestige-tyre-a.neto.com.au,9.116421923264706,0.6136854534811207
2019-10-15 00:37:53.712119,N061981,nunawadingtoyot.neto.com.au,9.116421923264706,0.6052243471818596
2019-10-15 00:37:53.71